In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os
import sys
from torchvision import transforms


project_root = os.getcwd()  
sys.path.append(os.path.join(project_root, "models"))
from dataset import *
from models.unet import UNet

In [ ]:
path = "./data"

# load datasets
train_set = DFC20(path, 
                  subset="val", 
                  use_s1=False, 
                  use_s2_RGB=True, 
                  use_s2_hr=False, 
                  use_s2_all=False,
                  as_tensor=True)

n_inputs = train_set.n_inputs

# set up dataloaders
train_loader = DataLoader(train_set,
                            batch_size=8,
                            shuffle=True,
                            pin_memory=True,
                            drop_last=False)

model = UNet(n_channels=n_inputs)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = model.to(device)

[Load]: 100%|██████████| 684/684 [00:00<00:00, 85518.94it/s]

loaded 684 samples from the DFC20 subset val


In [3]:
train_set.__getitem__(0)['label']

tensor([[6, 6, 6,  ..., 6, 6, 6],
        [6, 6, 6,  ..., 6, 6, 6],
        [6, 6, 6,  ..., 6, 6, 6],
        ...,
        [1, 1, 1,  ..., 4, 6, 6],
        [1, 1, 1,  ..., 4, 6, 6],
        [1, 1, 1,  ..., 4, 6, 6]])

In [ ]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
# try couple params & add validation loss
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_accuracy = 0.0
    num_batches = 0

    for batch in train_loader:
        inputs = batch['image'].to(device)
        labels = batch['label'].to(device)

        labels = labels - 1  # labels in range 0-9

        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Compute accuracy
        predicted_labels = torch.argmax(outputs, dim=1)
        accuracy = (predicted_labels == labels).float().mean().item() * 100

        total_accuracy += accuracy
        num_batches += 1

    avg_loss = running_loss / len(train_loader)
    avg_accuracy = total_accuracy / num_batches

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.2f}%")

# Save the model
torch.save(model.state_dict(), "unet_model.pth")


IndexError: Target 10 is out of bounds.